# Inserting necessary libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
tensorflow.__version__

'2.3.0'

In [3]:
tensorflow.keras.__version__

'2.4.0'

# Image Data Augumentation

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing Data Augumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'C:\Users\sujana\Desktop\project1\dataset\train_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'C:\Users\sujana\Desktop\project1\dataset\test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [7]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone1': 0, 'Earthquake1': 1, 'Flood1': 2, 'Wildfire1': 3}


In [8]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

# Creating the Model

In [9]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2

In [10]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0

# Compiling the Model

In [11]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fitting the Model

In [12]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=10, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
149/149 [==============================] - 28s 190ms/step - loss: 1.2256 - accuracy: 0.4259 - val_loss: 1.2616 - val_accuracy: 0.5101
Epoch 2/10
149/149 [==============================] - 27s 184ms/step - loss: 1.1000 - accuracy: 0.5350 - val_loss: 1.1450 - val_accuracy: 0.5101
Epoch 3/10
149/149 [==============================] - 28s 185ms/step - loss: 0.8940 - accuracy: 0.6159 - val_loss: 1.0122 - val_accuracy: 0.5657
Epoch 4/10
149/149 [==============================] - 27s 183ms/step - loss: 0.8516 - accuracy: 0.6348 - val_loss: 1.0739 - val_accuracy: 0.5253
Epoch 5/10
149/149 [==============================] - 28s 185ms/step - loss: 0.7468 - accuracy: 0.6685 - val_loss: 0.7308 - val_accuracy: 0.6970
Epoch 6/10
149/149 [==============================] - 28s 186ms/step - loss: 0.6965 - accuracy: 0.7156 - val_loss: 0.7787 - val_accuracy: 0.6515
Epoch 7/10
149/149 [==============================] - 

# Saving the Model

In [13]:
# Save the model
classifier.save('disaster.h5')

In [14]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# Predicting Results

In [15]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("disaster.h5") #loading the model for testing

In [17]:
img = image.load_img(r"C:\Users\sujana\Desktop\project1\dataset\test_set\Cyclone1\921.jpg",grayscale=False,
                          target_size= (64,64))#loading of the image\n
x = image.img_to_array(img)#image to array\n",
x = np.expand_dims(x,axis = 0)#changing the shape\n",
pred = model.predict_classes(x)#predicting the classes\n",
pred

array([0], dtype=int64)

In [18]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

'Cyclone'

In [22]:
!pip install jupyterthemes as jt

  Using cached jupyterthemes-0.20.0-py2.py3-none-any.whl (7.0 MB)


ERROR: Could not find a version that satisfies the requirement as
ERROR: No matching distribution found for as


In [23]:
!jt -t monokai

'jt' is not recognized as an internal or external command,
operable program or batch file.
